<a href="https://colab.research.google.com/github/Nimesh-Patel/Agent-Based-Modeling/blob/master/Copy_of_TextClass_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Classifier for NIOSH competition**

**Nim Pat**

**11.21.2019**



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
###  FIRST PART, Run this First and Upload Files 
from google.colab import files
uploaded = files.upload()

In [0]:
### Run Model

# import libraries
from fastai import *
from fastai.text import *

## import files
import io
df_train = pd.read_csv(io.BytesIO(uploaded['Train_Clean.csv']), encoding='iso-8859-1')
df_val = pd.read_csv(io.BytesIO(uploaded['Train_Clean.csv']), encoding='iso-8859-1')
df_test = pd.read_csv(io.BytesIO(uploaded['test_Clean.csv']), encoding='iso-8859-1')

## language model

# data for language model
data_lm = TextLMDataBunch.from_df(train_df = df_train, valid_df = df_val, path="")

# train language model
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)
learn_lm.lr_find(start_lr = slice(10e-7, 10e-5), end_lr=slice(0.1, 10))
learn_lm.recorder.plot(suggestion=True)
best_lm_lr = learn_lm.recorder.min_grad_lr
learn_lm.fit_one_cycle(10, best_lm_lr)
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, best_lm_lr)
print("Finish Train Lang model")
# save encoder
learn_lm.save_encoder('ft_enc')

## classifier
print("Starting classifer data import")
# data for classifier
data_clas = TextClasDataBunch.from_df(path="", train_df = df_train, valid_df = df_val, test_df=df_test, vocab=data_lm.train_ds.vocab, bs=32)
print("Starting Train classifer")
# train classifier
learn_clas = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.7)
learn_clas.load_encoder('ft_enc')
learn_clas.lr_find(start_lr=slice(10e-7, 10e-5), end_lr=slice(0.1, 10))
learn_clas.recorder.plot(skip_end=1, suggestion=True)
best_clf_lr = learn_clas.recorder.min_grad_lr
learn_clas.fit_one_cycle(10, best_clf_lr)
learn_clas.freeze_to(-2)
learn_clas.fit_one_cycle(10, slice(best_clf_lr/2., best_clf_lr))
learn_clas.unfreeze()
learn_clas.fit_one_cycle(10, slice(best_clf_lr/100, best_clf_lr))

## predictions

# generate predictions
preds, targets = learn_clas.get_preds(DatasetType.Test, ordered=True)
predictions = np.argmax(preds, axis = 1)
test_preds=predictions.tolist()
preds_df=pd.DataFrame(test_preds)

# re-label
outcomes=list(set(df_train['label']))
di={}
i=0
for o in outcomes:
  di[i]=o
  i=i+1
  
new_preds=preds_df.replace(di) 


# export predictions
new_preds.to_csv('preds.csv', index=False, header=False)
files.download('preds.csv')